In [41]:
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense, LSTM, Dropout # type: ignore
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from keras import backend as K

def reset_keras_session():
    K.clear_session()
    tf.compat.v1.reset_default_graph()
    np.random.seed(42)
    tf.random.set_seed(42)
    random.seed(42)

def prepare_data(data, target):
    columns = ['下月节假日数量', '本月平均高温', '本月平均低温']
    df = data[[target] + columns].copy()

    # 新数据行的字典，保持与原始数据结构一致
    new_data = {
        target: [0],
        '下月节假日数量': [0],
        '本月平均高温': [30],
        '本月平均低温': [20],
    }
    # 创建新数据行的 DataFrame，并确保列名与原始数据一致
    new_row = pd.DataFrame(new_data)
    # 将新行添加到原始数据中
    df = pd.concat([df, new_row], ignore_index=True)

    # 添加本月节假日数量列
    df['本月节假日数量'] = df['下月节假日数量'].shift(1)
    features = ['本月节假日数量']  # 特征列

    # 归一化
    scaler_features = MinMaxScaler()
    scaler_target = MinMaxScaler()
    feature_data = df[features]
    target_data = df[[target]]
    df_scaled_features = pd.DataFrame(scaler_features.fit_transform(feature_data), columns=feature_data.columns)
    df_scaled_target = pd.DataFrame(scaler_target.fit_transform(target_data), columns=target_data.columns)
    df_scaled = pd.concat([df_scaled_features, df_scaled_target], axis=1)
    
    # df_scaled = pd.DataFrame(df)

    # 添加增长率列
    df_scaled['增长率'] = df[target].pct_change().fillna(0)
    df_scaled['增长率'] = df_scaled['增长率'].replace([np.inf, -np.inf], 0)

    # 将过去n条用电量作为特征
    n = 1
    for i in range(1, n + 1):
        column_name = f'{target}_shift_{i}'
        features.append(column_name)
        df_scaled[column_name] = df_scaled[target].shift(i)

    # 将过去m条增长率作为特征
    m = 4
    for i in range(1, m + 1):
        column_name = f'增长率_shift_{i}'
        features.append(column_name)
        df_scaled[column_name] = df_scaled['增长率'].shift(i)

    # 删除包含NaN值的行
    df_scaled = df_scaled.dropna()

    X_train = df_scaled[features][:-1]
    y_train = df_scaled[target][:-1]
    X_pred = df_scaled[features][-1:]
    return X_train, y_train, X_pred, scaler_target

def build_model(input_shape):
    reset_keras_session()
    model = Sequential()
    model.add(LSTM(100, activation='relu', input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mae')
    return model

def predict_dailigoudian(train_data, target):
    X_train, y_train, X_pred, scaler = prepare_data(train_data, target)

    input_shape=(X_train.shape[1], 1)
    # 构建LSTM模型
    model = build_model(input_shape)
    
    # 训练模型
    model.fit(X_train, y_train, epochs=100, batch_size=4, verbose=0)
    
    # 预测
    y_pred = model.predict(X_pred, verbose=0)
    
    # 逆归一化预测结果
    y_pred_inv = scaler.inverse_transform(y_pred)  # 逆归一化
    return y_pred_inv[0][0]

# data = pd.read_csv('0612/data/代理购电.csv')
# target = '开发区供电中心'
# y_pred= predict_dailigoudian(data, target)
# print(y_pred)

In [42]:
def calculate(data, target):
    # 训练数据
    train_data = data[:-1]
    
    # 预测
    y_pred = predict_dailigoudian(train_data, target)

    # 实际值
    y_real = data[target].iloc[-1]

    # 预测的月份
    last_date = pd.to_datetime(train_data['date'].iloc[-1])
    predict_month = last_date + pd.DateOffset(months=1)
    predict_month_str = predict_month.strftime('%Y年%m月')
    acc = (1 - abs(y_pred - y_real) / y_real) * 100
    print(f'{predict_month_str} {target} 预测结果：{y_pred}，实际值：{y_real}，准确率：{acc:.2f}%')

def cal_more(data, target):
    calculate(data, target)
    for i in range(1, 5):
        calculate(data[:-i], target)

In [43]:
data = pd.read_csv('0612/data/代理购电.csv')
targets = ['客户服务中心', '开发区供电中心', '东营港供电中心', '黄三角农高区供电中心',
          '广饶县供电公司', '利津县供电公司','垦利区供电公司', '河口区供电公司', '东营区供电公司']

for target in targets:
    cal_more(data, target)
    print()

2024年05月 客户服务中心 预测结果：6047.8466796875，实际值：6565.15，准确率：92.12%
2024年04月 客户服务中心 预测结果：834.213134765625，实际值：5922.1464，准确率：14.09%
2024年03月 客户服务中心 预测结果：756.294189453125，实际值：863.2033，准确率：87.61%
2024年02月 客户服务中心 预测结果：827.268798828125，实际值：891.9601，准确率：92.75%
2024年01月 客户服务中心 预测结果：1173.3492431640625，实际值：979.8412，准确率：80.25%

2024年05月 开发区供电中心 预测结果：3424.23095703125，实际值：4174.188，准确率：82.03%
2024年04月 开发区供电中心 预测结果：6623.06884765625，实际值：3528.9584，准确率：12.32%
2024年03月 开发区供电中心 预测结果：5222.84521484375，实际值：5871.6114，准确率：88.95%
2024年02月 开发区供电中心 预测结果：6938.18798828125，实际值：5683.0449，准确率：77.91%
2024年01月 开发区供电中心 预测结果：7141.2509765625，实际值：7391.4372，准确率：96.62%

2024年05月 东营港供电中心 预测结果：1010.9547119140625，实际值：1375.252，准确率：73.51%
2024年04月 东营港供电中心 预测结果：1209.5682373046875，实际值：1053.7713，准确率：85.22%
2024年03月 东营港供电中心 预测结果：1123.926513671875，实际值：1198.9629，准确率：93.74%
2024年02月 东营港供电中心 预测结果：1136.3778076171875，实际值：1099.7067，准确率：96.67%
2024年01月 东营港供电中心 预测结果：1212.054443359375，实际值：1344.2361，准确率：90.17%

2024年05月 黄三角农高区供电中心 预测结果：489.926208496093